In [1]:
import numpy as np
import sympy as sym
import math as m

In [2]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [3]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    error = 1.
    it = 0
    while error >= precision and it < itmax:
        try:
            xn1 = xn - f(xn)/df(xn)
            error = np.abs(f(xn)/df(xn))
        except ZeroDivisionError:
            print('Zero Division')
        xn = xn1
        it += 1
    if it == itmax:
        return False
    else:
        return xn

In [4]:
def GetRoots(f,df,x,tolerancia = 10):
    Roots = np.array([])
    for i in x:
        root = GetNewton(f,df,i)
        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            if croot not in Roots:
                Roots = np.append(Roots, croot)
    Roots.sort()
    return Roots

In [5]:
def GetHermite(n, x):
    if n == 0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = ((2*x)*GetHermite(n-1,x)-2*(n-1)*GetHermite(n-2,x))
    return sym.expand(poly,x)

In [6]:
def GetDHermite(n,x):
    Pn = GetHermite(n,x)
    return sym.diff(Pn,x,1)

In [7]:
def GetAllRootsGHer (n):
    xn = np.linspace(-np.sqrt(4*n+1),np.sqrt(4*n+1),100)

    Hermite = GetHermite(n,x)
    DHermite = GetDHermite(n,x)
    
    poly = sym.lambdify([x],Hermite,'numpy')
    Dpoly = sym.lambdify([x],DHermite,'numpy')
    
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [8]:
def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)
    
    Hermite = (GetHermite(n-1,x))
    
    poly = sym.lambdify([x],Hermite,'numpy')
    Weights = ((2**(n-1))*m.factorial(n)*np.sqrt(np.pi))/((n)**2*poly(Roots)**2)
    
    return Weights

In [9]:
GetAllRootsGHer(20)

array([-5.38748089, -4.60368245, -3.94476404, -3.34785457, -2.78880606,
       -2.254974  , -1.73853771, -1.23407622, -0.73747373, -0.24534071,
        0.24534071,  0.73747373,  1.23407622,  1.73853771,  2.254974  ,
        2.78880606,  3.34785457,  3.94476404,  4.60368245,  5.38748089])

In [10]:
GetWeightsGHer(20)

array([2.22939365e-13, 4.39934099e-10, 1.08606937e-07, 7.80255648e-06,
       2.28338636e-04, 3.24377334e-03, 2.48105209e-02, 1.09017206e-01,
       2.86675505e-01, 4.62243670e-01, 4.62243670e-01, 2.86675505e-01,
       1.09017206e-01, 2.48105209e-02, 3.24377334e-03, 2.28338636e-04,
       7.80255648e-06, 1.08606937e-07, 4.39934099e-10, 2.22939365e-13])

In [37]:
def GetHermite(n,x,y):
      
    y = sym.exp(-x**2)
    
    poly = (-1)**n*sym.exp(x**2)*sym.diff(y,x,n)
    
    return poly

In [93]:
Hermite = []
n = 1
for i in range(n+1):
    Poly = GetHermite(i,x,y)
    Hermite.append(Poly)
for i in range(1, len(Hermite)):
    Hermite[i-1]=Hermite[i]
a = Hermite[-1]
Hermite.remove(a)

In [94]:
def funcion(roots, n, Hermite):
    return ((1/(np.sqrt(2**n)*m.factorial(n)))) * (1/(np.pi**(1/4)))*(roots)*(np.e**((-roots**2/2)))*Hermite

In [95]:
def Integral(funcion, roots, n, Hermite):
    return (np.abs(funcion(roots, n, Hermite))**2)*(roots**2)

In [96]:
Roots, Weights = np.polynomial.hermite.hermgauss(n)
def z (funcion, roots, n, Weights, Hermite):
    return sum(Integral(funcion, roots, n, Hermite)*Weights)

In [97]:
z(funcion, Roots, n, Weights, Hermite)

0